In [1]:
from bsm import bs_price, greeks, implied_vol_newton

S, K = 200.0, 210.0
T = 30/365
r, q = 0.05, 0.00
sigma = 0.20

price = bs_price(S, K, T, r, sigma, "call", q)
g = greeks(S, K, T, r, sigma, "call", q)
price, g


(np.float64(1.4615811587707057),
 {'delta': np.float64(0.2264536760489173),
  'gamma': np.float64(0.02624838509120609),
  'vega': np.float64(17.25921211476565),
  'theta_per_year': np.float64(-23.19016577551551),
  'rho': np.float64(3.602396223370911)})

In [2]:
true_sigma = 0.30
market_px = bs_price(S, K, T, r, true_sigma, "call", q)
iv = implied_vol_newton(market_px, S, K, T, r, "call", q)
true_sigma, market_px, iv

(0.3, np.float64(3.3730051848583997), np.float64(0.3000000010254372))

In [3]:
import yfinance as yf
import pandas as pd

tkr = yf.Ticker("AAPL")
hist = tkr.history(period="5d", interval="1d")
S_aapl = float(hist["Close"].iloc[-1])

divs = tkr.dividends
one_year_ago = pd.Timestamp.utcnow().tz_localize(None) - pd.Timedelta(days=365)
divs_last_year = divs[divs.index.tz_localize(None) >= one_year_ago]
div_sum = float(divs_last_year.sum()) if not divs_last_year.empty else 0.0
q_aapl = div_sum / S_aapl if S_aapl > 0 else 0.0

S_aapl, q_aapl

(229.35000610351562, 0.004403749610297125)

In [4]:
S = S_aapl
K = round(S_aapl, 0)  # near the money
T = 30/365
r = 0.05
q = q_aapl
sigma = 0.25

px = bs_price(S, K, T, r, sigma, "put", q)
greeks(S, K, T, r, sigma, "put", q), px

({'delta': np.float64(-0.4562643116788659),
  'gamma': np.float64(0.024115676122955697),
  'vega': np.float64(26.065457591773864),
  'theta_per_year': np.float64(-34.57225965548327),
  'rho': np.float64(-9.090055978304335)},
 np.float64(5.951458401005155))

In [5]:
exps = tkr.options
expiry = exps[0]  # nearest weekly
chain = tkr.option_chain(expiry)
calls = chain.calls[["strike", "bid", "ask", "lastPrice", "impliedVolatility"]].dropna()

calls["dist"] = (calls["strike"] - S).abs()
row = calls.sort_values("dist").iloc[0]

strike_mtm = float(row["strike"])
mid = float((row["bid"] + row["ask"]) / 2.0) if row["bid"] > 0 and row["ask"] > 0 else float(row["lastPrice"])
iv_yf = float(row["impliedVolatility"])

expiry_dt = pd.to_datetime(expiry).tz_localize("UTC")
today = pd.Timestamp.utcnow().normalize()
T_chain = max((expiry_dt - today).days, 1) / 365.0

iv_me = implied_vol_newton(mid, S, strike_mtm, T_chain, r, "call", q)
{
    "S": S, "K": strike_mtm, "market_mid": mid,
    "T_years": T_chain, "iv_yfinance": iv_yf, "iv_me": iv_me
}

{'S': 229.35000610351562,
 'K': 230.0,
 'market_mid': 3.25,
 'T_years': 0.0136986301369863,
 'iv_yfinance': 0.3078682495117187,
 'iv_me': np.float64(0.3262603717244667)}

In [12]:
from datetime import date

tkr = yf.Ticker("AAPL")
exps = tkr.options
expiry = date(2025, 8, 15)
print(expiry)
print(type(expiry))
# chain = tkr.option_chain(expiry)
# calls = chain.calls[["strike","bid","ask","lastPrice","impliedVolatility"]].dropna().copy()


2025-08-15
<class 'datetime.date'>


In [22]:
exps = tkr.options
expiry = exps[2]
chain = tkr.option_chain(expiry)
calls = chain.calls[["strike", "bid", "ask", "lastPrice", "impliedVolatility"]].dropna().copy()



In [47]:
df = pd.read_excel( 
    r"C:\Users\tanzi\Downloads\option-chain-ED-RELIANCE-28-Aug-2025.xlsx", 
    header=[0,1], 
    sheet_name="option-chain-ED-RELIANCE-28-Aug")

print(df.columns.to_list())

[('CALLS', 'OI'), ('CALLS', 'CHNG IN OI'), ('CALLS', 'VOLUME'), ('CALLS', 'IV'), ('CALLS', 'LTP'), ('CALLS', 'CHNG'), ('CALLS', 'BID QTY'), ('CALLS', 'BID'), ('CALLS', 'ASK'), ('CALLS', 'ASK QTY'), ('STRIKE', 'STRIKE'), ('PUTS', 'BID QTY'), ('PUTS', 'BID'), ('PUTS', 'ASK'), ('PUTS', 'ASK QTY'), ('PUTS', 'CHNG'), ('PUTS', 'LTP'), ('PUTS', 'IV'), ('PUTS', 'VOLUME'), ('PUTS', 'CHNG IN OI'), ('PUTS', 'OI')]


In [48]:
df = df.set_index(("STRIKE", "STRIKE"))

In [49]:
df.head()

CALLS                                                   \
                    OI CHNG IN OI VOLUME IV    LTP CHNG BID QTY     BID   
(STRIKE, STRIKE)                                                          
1140                 -          -      -  -      -    -    2000  235.15   
1160                 -          -      -  -      -    -    2000  215.45   
1180                68          -      5  -    200    5     500  198.10   
1200              1280          -     16  -  180.7 -0.3     500  178.45   
1220               118          -      -  -      -    -    5000  156.40   

                                    PUTS                                  \
                     ASK ASK QTY BID QTY   BID   ASK ASK QTY  CHNG   LTP   
(STRIKE, STRIKE)                                                           
1140              244.15    2000    1000   0.2  0.35    1500  -0.2  0.25   
1160              225.00    2000    1000   0.5  0.75   10000 -0.05   0.5   
1180              201.25     500     500   0.5   0.6     500 -0.05  0.55   
1200              181.25     500    2500  0.85   0.9    5500   0.1   0.9   
1220              164.85    5000   11000   0.8  1.05     500  -0.1   0.8   

                                                 
                     IV VOLUME CHNG IN OI    OI  
(STRIKE, STRIKE)                                 
1140              38.54     15          -    15  
1160              39.32     77          1   266  
1180              36.02    101        -26   219  
1200              35.33   1126        -79  2840  
1220              31.27    145        -52   638

In [51]:
calls_df = df['CALLS']
calls_df.head()

,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
"(STRIKE, STRIKE)",,,,,,,,,,
1140,-,-,-,-,-,-,2000,235.15,244.15,2000
1160,-,-,-,-,-,-,2000,215.45,225.00,2000
1180,68,-,5,-,200,5,500,198.10,201.25,500
1200,1280,-,16,-,180.7,-0.3,500,178.45,181.25,500
1220,118,-,-,-,-,-,5000,156.40,164.85,5000


In [52]:
print(calls_df.columns.to_list())

['OI', 'CHNG IN OI', 'VOLUME', 'IV', 'LTP', 'CHNG', 'BID QTY', 'BID', 'ASK', 'ASK QTY']


In [ ]:
calls_df = calls_df.reset_index()

calls_df.head()

,"(STRIKE, STRIKE)",OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
0,1140,-,-,-,-,-,-,2000,235.15,244.15,2000
1,1160,-,-,-,-,-,-,2000,215.45,225.00,2000
2,1180,68,-,5,-,200,5,500,198.10,201.25,500
3,1200,1280,-,16,-,180.7,-0.3,500,178.45,181.25,500
4,1220,118,-,-,-,-,-,5000,156.40,164.85,5000


In [65]:
calls_df = calls_df.rename(columns={ calls_df.columns[0]: "STRIKE" })
print(calls_df.columns.to_list())

['STRIKE', 'OI', 'CHNG IN OI', 'VOLUME', 'IV', 'LTP', 'CHNG', 'BID QTY', 'BID', 'ASK', 'ASK QTY']


In [66]:
calls_df.head()

,STRIKE,OI,CHNG IN OI,VOLUME,IV,LTP,CHNG,BID QTY,BID,ASK,ASK QTY
0,1140,-,-,-,-,-,-,2000,235.15,244.15,2000
1,1160,-,-,-,-,-,-,2000,215.45,225.00,2000
2,1180,68,-,5,-,200,5,500,198.10,201.25,500
3,1200,1280,-,16,-,180.7,-0.3,500,178.45,181.25,500
4,1220,118,-,-,-,-,-,5000,156.40,164.85,5000


In [67]:
calls_df.columns = ['_'.join(map(str, col)).strip().lower().replace(' ', '_') 
              if isinstance(col, tuple) else col.lower().replace(' ', '_') 
              for col in calls_df.columns]

print(calls_df.columns.to_list())

['strike', 'oi', 'chng_in_oi', 'volume', 'iv', 'ltp', 'chng', 'bid_qty', 'bid', 'ask', 'ask_qty']
